# AIRPORTS TABLE

In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import datetime
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Airports Table


The objetive is colect all usefull airports data in one only dataframe to be moreaccesible.
We will use two tables: 
- world-airports (obteined from the web ) it will be the base.
- airports extended (obtained from the web ourairports.com) --> we take the 'Tz database time zone' column
- country-code (obtained from datahub.io) --> we take columns 'ISO4217-currency_alphabetic_code' and 'ISO4217-currency_name'

First of all, we will read the three dataframes

In [62]:
#Get world_airports dataframe
world_airports = pd.read_csv('data/world-airports.csv')
world_airports

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
0,2434,EGLL,large_airport,London Heathrow Airport,51.470600,-0.461941,83.0,EU,United Kingdom,GB,...,London,1,EGLL,LHR,NaN,http://www.heathrowairport.com/,https://en.wikipedia.org/wiki/Heathrow_Airport,"LON, Londres",1251675,2019-10-19T10:16:49+00:00
1,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125.0,NaN,United States,US,...,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,NaN,1335475,2020-04-26T22:37:22+00:00
2,3754,KORD,large_airport,Chicago O'Hare International Airport,41.978600,-87.904800,672.0,NaN,United States,US,...,Chicago,1,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2018-09-16T02:35:35+00:00
3,3622,KJFK,large_airport,John F Kennedy International Airport,40.639801,-73.778900,13.0,NaN,United States,US,...,New York,1,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild",1052075,2021-04-10T17:02:47+00:00
4,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,1026.0,NaN,United States,US,...,Atlanta,1,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64011,324519,PL-0137,small_airport,Żłobek Mały Airfield,51.454611,23.542472,515.0,EU,Poland,PL,...,Żłobek,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2020-05-02T12:34:24+00:00
64012,42246,BITH,closed,Sauðanes Airport,66.251859,-15.273914,15.0,EU,Iceland,IS,...,Þórshöfn,0,BITH,NaN,NaN,NaN,https://en.wikipedia.org/wiki/%C3%9E%C3%B3rsh%...,"THO, Thorshofn",0,2021-03-23T00:07:10+00:00
64013,313231,HOO,small_airport,Nhon Co Airfield,11.978700,107.563800,2270.0,AS,Vietnam,VN,...,Đăk R'Lấp,0,NaN,HOO,NaN,NaN,NaN,NaN,0,2021-01-10T20:43:41+00:00
64014,323411,PL-0098,small_airport,Łagowiec,52.335250,15.665277,NaN,EU,Poland,PL,...,Łagowiec,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2017-05-24T02:23:15+00:00


In [63]:
world_airports.columns

Index(['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
       'elevation_ft', 'continent', 'country_name', 'iso_country',
       'region_name', 'iso_region', 'local_region', 'municipality',
       'scheduled_service', 'gps_code', 'iata_code', 'local_code', 'home_link',
       'wikipedia_link', 'keywords', 'score', 'last_updated'],
      dtype='object')

In [64]:
colu_list = ['Airport ID', 'Name', 'City', 'Country', 'iata_code', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 
             'Timezone', 'DST', 'Tz database time zone', 'Type', 'Source']

In [65]:
#Get airports_extended dataframe
airports_extended = pd.read_csv('data/airports-extended.csv', names = colu_list, header = None)
airports_extended

,Airport ID,Name,City,Country,iata_code,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12663,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports
12664,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports
12665,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports
12666,14110,Melitopol Air Base,Melitopol,Ukraine,\N,UKDM,46.880001,35.305000,0,\N,\N,\N,airport,OurAirports


In [66]:
#Get country_codes dataframe
country_codes = pd.read_csv('data/country-codes_csv.csv')
country_codes

,FIFA,Dial,ISO3166-1-Alpha-3,MARC,is_independent,ISO3166-1-numeric,GAUL,FIPS,WMO,ISO3166-1-Alpha-2,...,Sub-region Name,official_name_ru,Global Name,Capital,Continent,TLD,Languages,Geoname ID,CLDR display name,EDGAR
0,TPE,886,TWN,ch,Yes,158.0,925,TW,NaN,TW,...,NaN,NaN,NaN,Taipei,AS,.tw,"zh-TW,zh,nan,hak",1668284.0,Taiwan,NaN
1,AFG,93,AFG,af,Yes,4.0,1,AF,AF,AF,...,Southern Asia,Афганистан,World,Kabul,AS,.af,"fa-AF,ps,uz-AF,tk",1149361.0,Afghanistan,B2
2,ALB,355,ALB,aa,Yes,8.0,3,AL,AB,AL,...,Southern Europe,Албания,World,Tirana,EU,.al,"sq,el",783754.0,Albania,B3
3,ALG,213,DZA,ae,Yes,12.0,4,AG,AL,DZ,...,Northern Africa,Алжир,World,Algiers,AF,.dz,ar-DZ,2589581.0,Algeria,B4
4,ASA,1-684,ASM,as,Territory of US,16.0,5,AQ,NaN,AS,...,Polynesia,Американское Самоа,World,Pago Pago,OC,.as,"en-AS,sm,to",5880801.0,American Samoa,B5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,SAH,212,ESH,ss,In contention,732.0,268,WI,NaN,EH,...,Northern Africa,Западная Сахара,World,El-Aaiun,AF,.eh,"ar,mey",2461445.0,Western Sahara,U5
246,YEM,967,YEM,ye,Yes,887.0,269,YM,YE,YE,...,Western Asia,Йемен,World,Sanaa,AS,.ye,ar-YE,69543.0,Yemen,T7
247,ZAM,260,ZMB,za,Yes,894.0,270,ZA,ZB,ZM,...,Sub-Saharan Africa,Замбия,World,Lusaka,AF,.zm,"en-ZM,bem,loz,lun,lue,ny,toi",895949.0,Zambia,Y4
248,ZIM,263,ZWE,rh,Yes,716.0,271,ZI,ZW,ZW,...,Sub-Saharan Africa,Зимбабве,World,Harare,AF,.zw,"en-ZW,sn,nr,nd",878675.0,Zimbabwe,Y5


In [67]:
country_codes.columns

Index(['FIFA', 'Dial', 'ISO3166-1-Alpha-3', 'MARC', 'is_independent',
       'ISO3166-1-numeric', 'GAUL', 'FIPS', 'WMO', 'ISO3166-1-Alpha-2', 'ITU',
       'IOC', 'DS', 'UNTERM Spanish Formal', 'Global Code',
       'Intermediate Region Code', 'official_name_fr', 'UNTERM French Short',
       'ISO4217-currency_name', 'Developed / Developing Countries',
       'UNTERM Russian Formal', 'UNTERM English Short',
       'ISO4217-currency_alphabetic_code',
       'Small Island Developing States (SIDS)', 'UNTERM Spanish Short',
       'ISO4217-currency_numeric_code', 'UNTERM Chinese Formal',
       'UNTERM French Formal', 'UNTERM Russian Short', 'M49',
       'Sub-region Code', 'Region Code', 'official_name_ar',
       'ISO4217-currency_minor_unit', 'UNTERM Arabic Formal',
       'UNTERM Chinese Short', 'Land Locked Developing Countries (LLDC)',
       'Intermediate Region Name', 'official_name_es', 'UNTERM English Formal',
       'official_name_cn', 'official_name_en', 'ISO4217-currency_count

Lets clean the tables

In [ ]:
#Drop duplicates  world_airports
before = len(world_airports)
world_airports_clean = world_airports.drop_duplicates(subset=['ISO3166-1-Alpha-2'])
after = len(world_airports_clean)
print(f'Number of duplicate records dropped: {before - after}')

In [74]:
#Drop duplicates
before = len(country_codes)
country_codes_clean = country_codes.drop_duplicates(subset=['ISO3166-1-Alpha-2'])
after = len(country_codes_clean)
print(f'Number of duplicate records dropped: {before - after}')

Number of duplicate records dropped: 1


In [75]:
country_codes_clean.isnull().sum()

FIFA                                        12
Dial                                         1
ISO3166-1-Alpha-3                            0
MARC                                         4
is_independent                               0
ISO3166-1-numeric                            0
GAUL                                         6
FIPS                                         1
WMO                                         32
ISO3166-1-Alpha-2                            1
ITU                                         16
IOC                                         23
DS                                          27
UNTERM Spanish Formal                       53
Global Code                                  1
Intermediate Region Code                   142
official_name_fr                             1
UNTERM French Short                         53
ISO4217-currency_name                        2
Developed / Developing Countries             2
UNTERM Russian Formal                       53
UNTERM Englis

In [76]:
#Drop nulls
filter_na = country_codes_clean['ISO3166-1-Alpha-2'] != ''

In [77]:
country_codes_clean2 = country_codes_clean.dropna(subset=['ISO3166-1-Alpha-2'])

In [78]:
word_airports_1 = pd.merge(world_airports, country_codes_clean2[['ISO4217-currency_alphabetic_code', 'ISO4217-currency_name']], left_on= 'iso_country', right_on= 'ISO3166-1-Alpha-2', right_index=True, how = 'left')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [729]:
#Transform departure_hour and arrival_hour to datetime format
departure_hour = datetime.strptime(departure_h, "%m/%d/%Y %I:%M %p")
arrival_hour = datetime.strptime(arrival_h, "%m/%d/%Y %I:%M %p")
print(departure_hour)
print(arrival_hour)
print(departure_hour - arrival_hour)

2021-04-28 19:56:00
2021-04-28 09:56:00
10:00:00


In [730]:
diff = departure_hour - arrival_hour


In [731]:
#Print the result difference
#print("There are %s:%s hours of difference" % (diff.hour, diff.minute))
print(f"There are {diff} hours of difference")


There are 10:00:00 hours of difference


In [801]:
colu_list = ['Airport ID', 'Name', 'City', 'Country', 'iata_code', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 
             'Timezone', 'DST', 'Tz database time zone', 'Type', 'Source']

In [802]:
airports_extended = pd.read_csv('data/airports-extended.csv', names = colu_list)
airports_extended

,Airport ID,Name,City,Country,iata_code,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12663,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999,1670,\N,\N,\N,airport,OurAirports
12664,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000,85,\N,\N,\N,airport,OurAirports
12665,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports
12666,14110,Melitopol Air Base,Melitopol,Ukraine,\N,UKDM,46.880001,35.305000,0,\N,\N,\N,airport,OurAirports


In [803]:
filter_N = airports_extended['iata_code'] != r'\N'

In [804]:
airports_extended_filt = airports_extended[filter_N]
airports_extended_filt

,Airport ID,Name,City,Country,iata_code,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12653,14097,Bilogai-Sugapa Airport,Sugapa-Papua Island,Indonesia,UGU,WABV,-3.739560,137.031998,7348,\N,\N,\N,airport,OurAirports
12656,14100,Ramon Airport,Eilat,Israel,ETM,LLER,29.723694,35.011416,288,\N,\N,\N,airport,OurAirports
12657,14101,Rustaq Airport,Al Masna'ah,Oman,MNH,OORQ,23.640556,57.487500,349,\N,\N,\N,airport,OurAirports
12658,14102,Laguindingan Airport,Cagayan de Oro City,Philippines,CGY,RPMY,8.612203,124.456496,190,\N,\N,\N,airport,OurAirports


In [805]:
before = len(airports_extended_filt)
airports_clean = airports_extended_filt.drop_duplicates(subset=['iata_code'])
after = len(airports_clean)
print(f'Number of duplicate records dropped: {before - after}')

Number of duplicate records dropped: 0


In [747]:
def no_spaces(string):
    clean = string.strip()
    return clean

In [748]:
airports_clean['iata_code'] = airports_clean['iata_code'].apply(no_spaces)

<ipython-input-748-7f51ac3ceac2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airports_clean['iata_code'] = airports_clean['iata_code'].apply(no_spaces)


In [806]:
airports_extended_filt = airports_clean.astype({'iata_code': str})

In [807]:
world_airports = pd.read_csv('data/world-airports.csv')
world_airports

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
0,2434,EGLL,large_airport,London Heathrow Airport,51.470600,-0.461941,83.0,EU,United Kingdom,GB,...,London,1,EGLL,LHR,NaN,http://www.heathrowairport.com/,https://en.wikipedia.org/wiki/Heathrow_Airport,"LON, Londres",1251675,2019-10-19T10:16:49+00:00
1,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125.0,NaN,United States,US,...,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,NaN,1335475,2020-04-26T22:37:22+00:00
2,3754,KORD,large_airport,Chicago O'Hare International Airport,41.978600,-87.904800,672.0,NaN,United States,US,...,Chicago,1,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2018-09-16T02:35:35+00:00
3,3622,KJFK,large_airport,John F Kennedy International Airport,40.639801,-73.778900,13.0,NaN,United States,US,...,New York,1,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild",1052075,2021-04-10T17:02:47+00:00
4,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,1026.0,NaN,United States,US,...,Atlanta,1,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64011,324519,PL-0137,small_airport,Żłobek Mały Airfield,51.454611,23.542472,515.0,EU,Poland,PL,...,Żłobek,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2020-05-02T12:34:24+00:00
64012,42246,BITH,closed,Sauðanes Airport,66.251859,-15.273914,15.0,EU,Iceland,IS,...,Þórshöfn,0,BITH,NaN,NaN,NaN,https://en.wikipedia.org/wiki/%C3%9E%C3%B3rsh%...,"THO, Thorshofn",0,2021-03-23T00:07:10+00:00
64013,313231,HOO,small_airport,Nhon Co Airfield,11.978700,107.563800,2270.0,AS,Vietnam,VN,...,Đăk R'Lấp,0,NaN,HOO,NaN,NaN,NaN,NaN,0,2021-01-10T20:43:41+00:00
64014,323411,PL-0098,small_airport,Łagowiec,52.335250,15.665277,NaN,EU,Poland,PL,...,Łagowiec,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2017-05-24T02:23:15+00:00


In [794]:

world_airports.isnull().sum()

id                       0
ident                    1
type                     0
name                     0
latitude_deg             0
longitude_deg            0
elevation_ft         11980
continent            31254
country_name             0
iso_country            249
region_name              0
iso_region               0
local_region            16
municipality          5423
scheduled_service        0
gps_code             21820
iata_code            54751
local_code           32569
home_link            60760
wikipedia_link       53727
keywords             52351
score                    0
last_updated             0
dtype: int64

In [792]:
filter_na = world_airports['iata_code'] != ''

In [808]:
world_airports_filt = world_airports.dropna(subset=['iata_code'])
world_airports_filt

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
0,2434,EGLL,large_airport,London Heathrow Airport,51.470600,-0.461941,83.0,EU,United Kingdom,GB,...,London,1,EGLL,LHR,NaN,http://www.heathrowairport.com/,https://en.wikipedia.org/wiki/Heathrow_Airport,"LON, Londres",1251675,2019-10-19T10:16:49+00:00
1,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125.0,NaN,United States,US,...,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,NaN,1335475,2020-04-26T22:37:22+00:00
2,3754,KORD,large_airport,Chicago O'Hare International Airport,41.978600,-87.904800,672.0,NaN,United States,US,...,Chicago,1,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2018-09-16T02:35:35+00:00
3,3622,KJFK,large_airport,John F Kennedy International Airport,40.639801,-73.778900,13.0,NaN,United States,US,...,New York,1,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild",1052075,2021-04-10T17:02:47+00:00
4,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,1026.0,NaN,United States,US,...,Atlanta,1,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63966,308462,ZLZW,medium_airport,Zhongwei Shapotou Airport,37.573125,105.154454,8202.0,AS,China,CN,...,Zhongwei,1,ZLZW,ZHY,NaN,NaN,https://en.wikipedia.org/wiki/Zhongwei_Shapoto...,NaN,0,2013-04-10T00:28:33+00:00
63974,323879,CN-0051,small_airport,Zigong Airport,29.376484,104.625789,1133.0,AS,China,CN,...,Zigong,0,NaN,ZKL,NaN,NaN,NaN,Fengming General Airport,0,2017-06-15T09:36:39+00:00
63977,323222,HLZN,small_airport,Alzintan Airport,31.774878,12.250060,2080.0,AF,Libya,LY,...,Zintan,0,HLZN,ZIS,NaN,NaN,https://en.wikipedia.org/wiki/Alzintan_Airport,NaN,0,2017-05-14T04:53:16+00:00
64000,327355,ZUMT,medium_airport,Zunyi Maotai Airport,27.816380,106.332680,4068.0,AS,China,CN,...,Zunyi,1,ZUMT,WMT,NaN,NaN,https://en.wikipedia.org/wiki/Zunyi_Maotai_Air...,NaN,0,2018-04-26T10:22:37+00:00


In [809]:
before = len(world_airports_filt)
world_airports_clean = world_airports_filt.drop_duplicates(subset=['iata_code'])
after = len(world_airports_clean)
print(f'Number of duplicate records dropped: {before - after}')

Number of duplicate records dropped: 124


In [743]:
world_airports_clean['iata_code']

0        LHR
1        LAX
2        ORD
3        JFK
4        ATL
        ... 
63966    ZHY
63974    ZKL
63977    ZIS
64000    WMT
64013    HOO
Name: iata_code, Length: 9142, dtype: object

In [799]:
world_airports_clean = world_airports_filt.astype({'iata_code': str})

In [751]:
world_airports_clean['iata_code'] = world_airports_clean['iata_code'].apply(no_spaces)

In [810]:
word_airports_1 = pd.merge(world_airports_clean, airports_extended_filt['Tz database time zone'], on = 'iata_code', how = 'left')

KeyError: 'iata_code'

In [79]:
world_airports_extended = pd.read_csv('data/world-airports-extended.csv')
world_airports_extended

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated,Tz database time zone,ISO4217-currency_alphabetic_code,ISO4217-currency_name
0,2434,EGLL,large_airport,London Heathrow Airport,514.706,-461.941,83.0,EU,United Kingdom,GB,...,LHR,NaN,http://www.heathrowairport.com/,https://en.wikipedia.org/wiki/Heathrow_Airport,"LON, Londres",1251675,2019-10-19T10:16:49+00:00,Europe/London,GBP,Pound Sterling
1,3632,KLAX,large_airport,Los Angeles International Airport,33.942.501,-118.407.997,125.0,NaN,United States,US,...,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,NaN,1335475,2020-04-26T22:37:22+00:00,America/Los_Angeles,USD,US Dollar
2,3754,KORD,large_airport,Chicago O'Hare International Airport,419.786,-879.048,672.0,NaN,United States,US,...,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2018-09-16T02:35:35+00:00,America/Chicago,USD,US Dollar
3,3622,KJFK,large_airport,John F Kennedy International Airport,40.639.801,-737.789,13.0,NaN,United States,US,...,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild",1052075,2021-04-10T17:02:47+00:00,America/New_York,USD,US Dollar
4,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,336.367,-84.428.101,1026.0,NaN,United States,US,...,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00,America/New_York,USD,US Dollar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64011,324519,PL-0137,small_airport,Żłobek Mały Airfield,51.454.611,23.542.472,515.0,EU,Poland,PL,...,NaN,NaN,NaN,NaN,NaN,0,2020-05-02T12:34:24+00:00,\N,PLN,Zloty
64012,42246,BITH,closed,Sauðanes Airport,66.251.859,-15.273.914,15.0,EU,Iceland,IS,...,NaN,NaN,NaN,https://en.wikipedia.org/wiki/%C3%9E%C3%B3rsh%...,"THO, Thorshofn",0,2021-03-23T00:07:10+00:00,\N,ISK,Iceland Krona
64013,313231,HOO,small_airport,Nhon Co Airfield,11.9787,1.075.638,2270.0,AS,Vietnam,VN,...,HOO,NaN,NaN,NaN,NaN,0,2021-01-10T20:43:41+00:00,NaN,VND,Dong
64014,323411,PL-0098,small_airport,Łagowiec,5.233.525,15.665.277,NaN,EU,Poland,PL,...,NaN,NaN,NaN,NaN,NaN,0,2017-05-24T02:23:15+00:00,\N,PLN,Zloty


In [80]:
world_airports_extended.columns

Index(['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
       'elevation_ft', 'continent', 'country_name', 'iso_country',
       'region_name', 'iso_region', 'local_region', 'municipality',
       'scheduled_service', 'gps_code', 'iata_code', 'local_code', 'home_link',
       'wikipedia_link', 'keywords', 'score', 'last_updated',
       'Tz database time zone', 'ISO4217-currency_alphabetic_code',
       'ISO4217-currency_name'],
      dtype='object')